In [290]:
from pymongo import MongoClient, GEOSPHERE
from pandas.io.json import json_normalize
import statistics as stats
import pandas as pd
import os
import folium
import requests
from bs4 import BeautifulSoup
import time
import re

In [291]:
client = MongoClient ('localhost', 27017)
db = client['companies']

### Se buscan las diferentes categorías, para ver cuales utilizamos

In [292]:
categorias = db.companies.distinct('category_code');
print(list(categorias))

['enterprise', 'web', 'software', 'news', 'network_hosting', 'games_video', 'mobile', 'music', 'social', 'search', 'messaging', 'advertising', 'photo_video', 'security', 'finance', 'ecommerce', 'travel', 'hardware', 'public_relations', 'other', 'real_estate', 'semiconductor', 'analytics', 'health', 'legal', 'sports', 'biotech', 'cleantech', 'education', 'consulting', 'transportation', None, 'hospitality', 'fashion', 'nonprofit', 'nanotech', 'automotive', 'design', 'manufacturing', 'government', 'local', 'medical']


### Se escoge una lista de categorías y los campos que vamos a obtener de la base de datos

In [293]:
categorias = ['web', 'software', 'games_video', 'social', 'design', 'search', 'consulting']
campos = {'name': 1,
          'number_of_employees': 1,
          'category_code': 1,
          'acquisition': 1,
          'founded_year': 1,
          'funding_rounds': 1,
          'offices.latitude': 1,
          'offices.longitude': 1,
          'offices.city': 1,
          'offices.country_code': 1}

In [294]:
def buscar_oficinas(categorias, campos, anyo, mm, num_emp):
    '''
    Para realizar la búsqueda en la base de datos
    
    categorias: los item por los que filtrar
    campos: los campos a mostrar
    mm: par el número de empleados, si es mayor o menor $gt, $lt ...
    num_emp: número de empleados
    '''
    datos = db.companies.find({'$and':[{'category_code': {'$in':categorias}},
                                       {'number_of_employees': {mm: num_emp}},
                                       {'offices': {'$exists': True, '$not': {'$size': 0}}},
                                       {'deadpooled_year': None},
                                       {'founded_year':{'$gte': anyo}}]},
                              campos)
    return datos



### Se realizan dos búsquedas una para startup y otra para empresas grandes

In [295]:
cursor_big = buscar_oficinas(categorias, campos, 2000, '$gt', 50)
cursor_startup = buscar_oficinas(categorias, campos, 2009, '$lt', 50)

#### Normalizamos el campo offices tanto para las startup como para las big

In [296]:
other_office_data = json_normalize(data = cursor_big, 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
other_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,Menlo Park,USA,37.416050,-122.151801,Facebook,social,2004,5299
1,Dublin,IRL,53.344104,-6.267494,Facebook,social,2004,5299
2,New York,USA,40.755716,-73.979247,Facebook,social,2004,5299
3,San Francisco,USA,37.776805,-122.416924,Twitter,social,2006,1300
4,San Francisco,USA,37.778613,-122.395289,Powerset,search,2006,60


In [297]:
startup_office_data = json_normalize(data = cursor_startup, 
                             record_path ='offices', 
                             meta=['name', 'category_code', 'founded_year', 'number_of_employees']) 
startup_office_data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees
0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20
1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8
2,Hannover,DEU,NaN,NaN,headr,web,2012,8
3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30
4,Norderstedt,DEU,53.707739,10.023246,alluc,games_video,2009,7


### Se concatenan ambos dataframes y se eliminan los registros con nulos

In [298]:
data = pd.concat([startup_office_data, other_office_data])
num_office = data['name'].value_counts() # Estos valores también los utilizaremos más adelante

def med_empleados(empleados, oficinas):
    return empleados / oficinas

data['med_employees'] = data.apply(lambda x: x['number_of_employees'] / num_office[x['name']], axis = 1)
data.head()

,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,med_employees
0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,20.0
1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8,4.0
2,Hannover,DEU,NaN,NaN,headr,web,2012,8,4.0
3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30,30.0
4,Norderstedt,DEU,53.707739,10.023246,alluc,games_video,2009,7,7.0


### Se eliminan los nulos

In [299]:
data = data.dropna()
data = data.reset_index()
data.shape

(551, 10)

### Se juntan en un único campo tanto la longitud como la latitud, para pasarlo a mongo y utilizar geolocation

In [300]:
data['coordenadas'] = [[x, y] for x, y in zip(data['longitude'], data['latitude'])]
data.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,med_employees,coordenadas
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,20.0,"[-73.985506, 40.757929]"
1,1,Berlin,DEU,52.501345,13.410907,headr,web,2012,8,4.0,"[13.4109071, 52.5013449]"
2,3,San Mateo,USA,37.566879,-122.323895,Fixya,web,2013,30,30.0,"[-122.323895, 37.566879]"
3,4,Norderstedt,DEU,53.707739,10.023246,alluc,games_video,2009,7,7.0,"[10.023246, 53.707739]"
4,5,West Dundee,USA,42.091078,-88.288749,Crootpad,games_video,2009,2,2.0,"[-88.288749, 42.091078]"


### Se guarda en un json el dataframe

In [301]:
data.to_json('oficinas.json', orient="records", lines= True)

### Se crea la base de datos, colección y se asigna el índice

In [302]:
db_ofi = client['oficinas']

def create_db(archivo = 'oficinas'):
    '''
    Crea la base de y la coleeción para oficinas
    '''
    
    existe = os.path.isfile(archivo + '.json')
    if existe:
        os.system('mongoimport --db oficinas --collection oficinas --drop --file '+ archivo +'.json')
        db_ofi.oficinas.create_index([('coordenadas', GEOSPHERE )])
        print('Base de datos y collección creadas')
    else:
        raise ValueError('Error archivo no encontrado')
    
create_db()

Base de datos y collección creadas


### Se realiza la búsqueda de las oficinas más cercanas para cada registro

In [303]:
def buscar_cercanas(lng, lat, min_dist_m = 0, max_dist_m = 3000):
    '''
    devuelve las oficinas más cercanas a un punto dado
    '''
    
    nearLocation = {
        "lng": lng,
        "lat": lat
    }
    busqueda = db_ofi.oficinas.find({
        "coordenadas": {
         "$near": {
           "$geometry": {
              "type": "Point" ,
              "coordinates": [ nearLocation["lng"] , nearLocation["lat"] ]
           },
           '$minDistance': min_dist_m,
           "$maxDistance": max_dist_m, 
         }
       }
        
    },{'latitude': 1,
       'longitude': 1,
       'city': 1,
       'name': 1,
       'category_code': 1,
       'number_of_employees': 1,
       '_id': 0})
    
    return busqueda   

In [304]:
data['cercanas'] = data.apply(lambda x : list(buscar_cercanas(x['longitude'], x['latitude'])), axis = 1)

def ratios_sb(oficinas):
    '''
    calcula el ratio de grandes empresas respecto a startup
    '''
    startup, big = 0, 0
    for office in oficinas:
        if office['number_of_employees'] > 250:
            big += 1
        else:
            startup += 1
    if startup > 0 and big > 0:
        salida = big / startup
    else:
        salida = 100 
    return salida


### Se añade la columna num_offices y el ratio, además se añade a cada dicionario de la columna cercanas la media de empleados por oficina

In [305]:
data['num_offices'] = data['cercanas'].apply(lambda x: len(x))
data['ratio'] = data['cercanas'].apply(ratios_sb)

datos = data['cercanas']
for i in range(len(datos)):
    for j in range(len(datos[i])):
        datos[i][j]['total_empleados'] = round((datos[i][j]['number_of_employees'] / num_office[datos[i][j]['name']]), 2)
            


### Se añade una una nueva columna con el total de empleados en las oficinas cercanas

In [306]:
def suma(oficinas):
    return sum(item['total_empleados'] for item in oficinas)
   
data['total_empleados_cercanas'] = data.apply(lambda x : suma(x['cercanas']), axis = 1)
data['cercanas'][4] # Ejemplo

[{'city': 'West Dundee',
  'latitude': 42.091078,
  'longitude': -88.288749,
  'name': 'Crootpad',
  'category_code': 'games_video',
  'number_of_employees': 2,
  'total_empleados': 2.0}]

### Se crea un dataframe para los puntos que tengan más de 9 oficinas a su alrededor

In [307]:
datos_finales = data[(data.num_offices > 9)].copy()
datos_finales.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,med_employees,coordenadas,cercanas,num_offices,ratio,total_empleados_cercanas
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,20.0,"[-73.985506, 40.757929]","[{'city': 'New York', 'latitude': 40.757929, '...",23,0.150000,3375.79
5,7,San Francisco,USA,37.789629,-122.399878,Macroaxis,software,2009,8,8.0,"[-122.3998782, 37.7896292]","[{'city': 'San Francisco', 'latitude': 37.7896...",39,0.344828,6114.75
6,8,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6,6.0,"[-122.387799, 37.760524]","[{'city': 'Santa Clara', 'latitude': 37.760524...",13,0.444444,2366.08
19,31,Paris,FRA,48.878562,2.360369,Stupeflix,web,2009,15,7.5,"[2.3603689, 48.8785618]","[{'city': 'Paris', 'latitude': 48.8785618, 'lo...",10,0.111111,672.00
20,34,New York,USA,40.744618,-73.987764,Yipit,web,2010,23,23.0,"[-73.987764, 40.744618]","[{'city': 'New York', 'latitude': 40.744618, '...",27,0.125000,3683.79


### Ahora vamos realizar un scraping de una web para obtener un precio de alquiler indicativo de que localización puede ser más cara a la hora de situar la empresa

In [308]:
def moda_ciudades(datos):
    '''
    devuleve la ciudad que más se repite
    en la columna cercana
    '''
    moda = []
    for c in datos:
        moda.append(c['city'])
        #max(set(moda), key = moda.count)
    return stats.mode(moda)

### Se cra una nueva columna con el nombre de esa ciudad, para utilizarla en el scraping

In [309]:
datos_finales['city_search'] = datos_finales.apply(lambda x : moda_ciudades(x['cercanas']), axis = 1)

### También se cambia el valor de esta nueva columna para adaptarla a la búsqueda

In [331]:
datos_finales.loc[datos_finales['city_search'] == 'New York', 'city_search'] = 'New York City'
datos_finales.loc[datos_finales['city_search'] == 'Mountain View', 'city_search'] = 'Mountain View California'
datos_finales.head()

,index,city,country_code,latitude,longitude,name,category_code,founded_year,number_of_employees,med_employees,coordenadas,cercanas,num_offices,ratio,total_empleados_cercanas,city_search,costo_alquiler
0,0,New York,USA,40.757929,-73.985506,PeekYou,search,2012,20,20.0,"[-73.985506, 40.757929]","[{'city': 'New York', 'latitude': 40.757929, '...",23,0.150000,3375.79,New York City,1754
5,7,San Francisco,USA,37.789629,-122.399878,Macroaxis,software,2009,8,8.0,"[-122.3998782, 37.7896292]","[{'city': 'San Francisco', 'latitude': 37.7896...",39,0.344828,6114.75,San Francisco,2071
6,8,Santa Clara,USA,37.760524,-122.387799,Fuzz,games_video,2011,6,6.0,"[-122.387799, 37.760524]","[{'city': 'Santa Clara', 'latitude': 37.760524...",13,0.444444,2366.08,San Francisco,2071
19,31,Paris,FRA,48.878562,2.360369,Stupeflix,web,2009,15,7.5,"[2.3603689, 48.8785618]","[{'city': 'Paris', 'latitude': 48.8785618, 'lo...",10,0.111111,672.00,Paris,1191
20,34,New York,USA,40.744618,-73.987764,Yipit,web,2010,23,23.0,"[-73.987764, 40.744618]","[{'city': 'New York', 'latitude': 40.744618, '...",27,0.125000,3683.79,New York City,1754


In [311]:
ciudades = set(datos_finales['city_search'])
ciudades

{'London',
 'Mountain View California',
 'New York City',
 'Paris',
 'San Francisco'}

In [312]:
def limpiar_precio(texto):
    temp = texto.split()
    precio = re.sub('\D', '', temp[0])
    return precio

def obtener_precio(ciudad):
    '''
    Realiza scraping y devuleve el precio 
    del alquiler de un estudio
    '''
    valor = dict()
    temp = ciudad.replace(' ', '-')
    url = 'https://www.expatistan.com/price/studio-rent-normal-area/'+temp+'/EUR' 
    
    numb_html = requests.get(url).text
    soup = BeautifulSoup(numb_html, "html.parser")
    
    coste = soup.find_all('span', class_= 'city-1')
    if coste:
        valor[ciudad] = limpiar_precio(coste[0].text)
    else:
        valor[ciudad] =  None
    time.sleep(2/1000)
    
    return valor

In [313]:
valores = {}
for c in ciudades:
    valores.update(obtener_precio(c))

In [314]:
valores

{'Paris': '1191',
 'New York City': '1754',
 'Mountain View California': '1967',
 'London': '1435',
 'San Francisco': '2071'}

### Se añade el valor al dataframe

In [315]:
datos_finales['costo_alquiler'] = datos_finales.apply(lambda x : valores[x['city_search']], axis = 1)

In [316]:
datos_finales['costo_alquiler'] = datos_finales['costo_alquiler'].astype('int64')
datos_finales.describe()

,index,latitude,longitude,founded_year,number_of_employees,med_employees,num_offices,ratio,total_empleados_cercanas,costo_alquiler
count,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000,91.000000
mean,244.604396,41.151819,-82.675999,2005.615385,240.373626,118.962977,27.186813,14.489241,3820.269890,1826.186813
std,223.278196,4.657100,44.514928,3.039624,657.351786,251.895584,10.997891,35.103205,2197.951226,264.469066
min,0.000000,37.400465,-122.419204,2000.000000,1.000000,1.000000,10.000000,0.066667,443.580000,1191.000000
25%,46.000000,37.787311,-122.399972,2003.000000,55.000000,21.500000,14.500000,0.125000,1749.105000,1754.000000
50%,172.000000,40.714353,-74.005973,2006.000000,85.000000,50.000000,27.000000,0.333333,3683.790000,1754.000000
75%,361.500000,40.751428,-73.983690,2008.000000,145.000000,100.000000,39.000000,0.350985,6114.750000,2071.000000
max,720.000000,51.524107,2.360369,2012.000000,5299.000000,1766.333333,40.000000,100.000000,6144.420000,2071.000000


### Se busca un punto con un ratio mayor o igual a 0.15 y cuyo coste sea menor que la media 
##### En este caso obtenemos un único resultado

In [371]:
final = datos_finales.cercanas[((datos_finales['ratio'] > float(0.15)) & 
                       (datos_finales['ratio'] < float(100.0))) & (datos_finales['costo_alquiler'] < 1826)]

data_final = pd.DataFrame(final.values[0])
data_final.head()

,category_code,city,latitude,longitude,name,number_of_employees,total_empleados
0,web,New York,40.761855,-73.983754,Cellfish,220,220.00
1,software,New York,40.764577,-73.979901,Unison Technologies,30,30.00
2,search,New York,40.757929,-73.985506,PeekYou,20,20.00
3,web,New York,40.756054,-73.986951,Udorse,3,3.00
4,social,New York,40.755716,-73.979247,Facebook,5299,1766.33


### Se calcula la latitud y longitud media respecto a las oficinas anteriores, para posicionar la nueva oficina

In [372]:
long_med = data_final['longitude'].mean()
lat_med = data_final['latitude'].mean()

nueva_ofi = ['games_video', '', lat_med, long_med, 'The new office', 50, 50]

data_final.loc[len(data_final)] = nueva_ofi
data_final[-1:]

,category_code,city,latitude,longitude,name,number_of_employees,total_empleados
21,games_video,,40.749214,-73.98684,The new office,50,50.0


### Mapa con la localización final de la oficina

In [354]:
loc_oficina = data_final[['latitude', 'longitude']][(data_final['name'] == 'The new office')]
loc_oficina.reset_index()

,index,latitude,longitude
0,21,40.749214,-73.98684


In [356]:
punto = [loc_oficina.iloc[0]['latitude'], loc_oficina.iloc[0]['longitude']]
mapa_final = folium.Map(location= punto, tiles="openstreetmap", zoom_start=15)

for i in range(0,len(data_final)):
    if  data_final.iloc[i]['name'] == 'The new office':
        folium.Marker([data_final.iloc[i]['latitude'], 
                       data_final.iloc[i]['longitude']], 
                      popup = (data_final.iloc[i]['name']),
                      icon = folium.Icon(color='blue')).add_to(mapa_final)
    else:
        folium.Marker([data_final.iloc[i]['latitude'], 
                       data_final.iloc[i]['longitude']], 
                      popup = data_final.iloc[i]['name'],
                      icon = folium.Icon(color='green')).add_to(mapa_final) 
        
leyenda = '''
    <style type="text/css">
        #leyenda{
            position: fixed;
            z-index: 9999;
            font-size: 1em;
            background-color: #ffffff;
            color: #333333;
            bottom: 10px;
            right: 10px;
            padding: 10px;
            border: 1px solid #333333;
        }
        
        #leyenda .verde{
            color: #71af26;
        }
        
        #leyenda .azul{
            color: #36a5d6;
        }
    
    </style>
    
     <div id="leyenda">
        <i class="fa fa-map-marker fa-2x azul"></i> The new office <br/> 
        <i class="fa fa-map-marker fa-2x verde"></i> Otras empresas    
      </div>
     '''
mapa_final.get_root().html.add_child(folium.Element(leyenda))        
        
mapa_final.save('map-final.html')         

### Se crea archivo para pasar a tableau

In [144]:
data_final.to_json('oficinas_tableau.json', orient="records", lines= True)

## Otros casos

#### Se busca la mayor concentración de empleados

In [346]:
final = datos_finales.cercanas[(datos_finales['total_empleados_cercanas'].idxmax())]
df_final = pd.DataFrame(final)
df_final.head()

,category_code,city,latitude,longitude,name,number_of_employees,total_empleados
0,web,San Francisco,37.766909,-122.406676,Flixster,75,75.00
1,search,San Francisco,37.766909,-122.406676,Trulia,500,500.00
2,games_video,San Francisco,37.765158,-122.404234,Zynga,115,28.75
3,games_video,San Francisco,37.770510,-122.401581,ngmoco,150,150.00
4,social,San Francisco,37.775300,-122.418600,Tagged,162,162.00


#### Se busca el ratio menor, ya que queremos la menor diferencia entre número de startup y grandes empresas

In [347]:
final = datos_finales.cercanas[(datos_finales['ratio'].idxmin())]
df_final = pd.DataFrame(final)
df_final.head()

,category_code,city,latitude,longitude,name,number_of_employees,total_empleados
0,games_video,New York,40.717248,-74.002662,Boonty,150,150.0
1,software,New York,40.718888,-74.001169,ReverbNation,85,42.5
2,software,New York,40.714353,-74.005973,Backbase,80,40.0
3,web,New York,40.725883,-74.006196,TheLadders,110,110.0
4,software,New York,40.707549,-74.005600,Vichara Technologies,100,50.0


#### Se muestra mapa con las localizaciones de todas las empresas 

In [282]:
mapa = folium.Map(location=[10, 0], tiles="openstreetmap", zoom_start=2)

for i in range(0,len(data)):
    if data.iloc[i]['number_of_employees'] <= 50:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='green')).add_to(mapa)
    if data.iloc[i]['number_of_employees'] > 50 and data.iloc[i]['number_of_employees'] < 249:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='red')).add_to(mapa)        
    elif data.iloc[i]['number_of_employees'] > 250:
        folium.Marker([data.iloc[i]['latitude'], 
                       data.iloc[i]['longitude']], 
                      popup=data.iloc[i]['name'],
                      icon=folium.Icon(color='blue')).add_to(mapa)  
        
        
        
leyenda = '''
    <style type="text/css">
        #leyenda{
            position: fixed;
            z-index: 9999;
            font-size: 1em;
            background-color: #ffffff;
            color: #333333;
            bottom: 10px;
            right: 10px;
            padding: 10px;
            border: 1px solid #333333;
        }
        
        #leyenda .verde{
            color: #71af26;
        }
        
        #leyenda .azul{
            color: #36a5d6;
        }
        
        #leyenda .naranja{
            color: #da4300;
        }        
    </style>
    
     <div id="leyenda">
        <i class="fa fa-map-marker fa-2x verde"></i> Startup <br/>
        <i class="fa fa-map-marker fa-2x naranja"></i> Mediana <br/>        
        <i class="fa fa-map-marker fa-2x azul"></i> Big Company
      </div>
     '''
mapa.get_root().html.add_child(folium.Element(leyenda))
mapa.save('map-oficinas.html')
